用户u对物品i感兴趣：
$$Preference(u,i) = r_{ui} = p_{u}^{T}q_{i} = \sum_{k=1}^{K}p_{u,k}q_{i,k}$$
$p_{u,k}$代表了用户u的兴趣和第k个隐类的关系，$q_{i,k}$代表了k个隐类和物品i的关系。

In [1]:
# 负样本采样
import random
def RandomSelectNegativeSample(items):
    '''
    item_pool: items for selecting,global
    items: positive samples of the user
    '''
    ret = {}
    for i in items.keys():
        ret[i] = 1
    n = 0
    for i in range(0,len(items)*3):
        item = item_pool[random.randint(0,len(item_pool)-1)]
        if item in ret:
            continue
        ret[item] = 0
        n += 1
        if n > len(items):
            break
    return ret

cost function:
$$C = \sum_{(u,i)\in K}(r_{ui}-\hat{r}_{ui})^{2} = \sum_{(u,i)\in K}(r_{ui}- \sum_{k=1}^{K}p_{u,k}q_{i,k})^{2} +\lambda \lVert P_{u}\rVert^{2}+\lambda \lVert q_{i}\rVert^{2}$$
通过对损失函数求偏导：  
利用梯度下降法：  
$$ p_{u,k} = p_{u,k} +\alpha(q_{i,k} - \lambda p_{u,k})$$
$$ q_{i,k} = q_{i,k} +\alpha(p_{u,k} - \lambda q_{i,k})$$

In [3]:
'''
F: 隐含特征数目
alpha： learning rate
lambd： regularization
ratio：正负样本的个数 这个参数指标对LFM影响最大
提高ratio可以增加准确率和召回率，但是覆盖率会降低，流行度会提高，也就是说ratio控制算法挖掘长尾的能力。
'''
def LatentFactorModel(user_items, F, N, alpha, lambd): # alpha是学习速率
    [P,Q] = InitModel(user_items, F) # F是分为多少类
    for step in range(0,N):
        for user, items in user_items.items():
            samples = RandomSelectNegativeSample(items)
            for item, rui in samples.items():
                eui = rui - Predict(user,item)
                for f in range(0,F):
                    P[user][f] += alpha * (eui*Q[item][f] - lambd * P[user][f])
                    Q[item][f] += alpha * (eui*P[user][f] - lambd * Q[item][f])
            alpha *= 0.9
    return P,Q
            
def Recommend(user,P,Q):
    rank = {}
    for f,puf in P[user].items():
        for i,qif in Q[item].items():
            if i not in rank:
                rank[i] += puf*qif
    return rank

LFM实时性方案
解决方案：  
1、利用新闻链接的内容属性（关键次、类别）得到链接i的内容$y_{i}$，实时收集用户的链接行为，利用这些数据得到链接i的隐含特征向量$q_{i}$.  
2、利用如下公式预测用户u是否会单机链接i：  


$$ r_{ui} = x_{u}^{T}y_{i} + p_{u}^{T}q_{i}$$  
$x_{u}$是用户向量，根据用户的历史行为记录获得，一天只需要计算一次，$x_{uk}$是用户u对内容特征k的兴趣程度，$y_{i}$是根据物品的内容属性直接生成的。  
### 基于图的模型  

基于随机游走的PersonalRank算法：   给用户u进行i个性化推荐，用户u可以从对应的节点$v_{u}$开始在用户物品二分图上进行随机游走。 游走到任意一个节点，首先按照概率$\alpha$决定是否继续游走， 还是停止这次游走，重新从$v_{u}$开始游走。 如果决定继续游走，就从当前节点只想的节点当中按照均匀分布随机选择一个节点作为游走下次经过的节点。这样， 经过很多次随机游走之后，每个物品节点访问的概率会收敛到一个数，最终的推荐列表种物品的权重就是物品节点的访问概率。（类似于马尔科夫状态度，最终收敛）。

$$ PR(v) = \begin{cases} \alpha \sum_{v^{'} \in (v)}  \frac{PR(v^{'})}{|out(v^{'})|} (v \neq v_{u}) \\ (1-\alpha) + \alpha \sum_{v^{'} \in (v)} \frac{PR(v^{'})}{|out(v^{'})|} (v = v_{u})\end{cases} $$

$$ M(v,v^{'}) = \frac{1}{|out(v^{'})|} $$
迭代公式变为：
$$ r = (1-\alpha)r_{0} + \alpha M^{T}r$$

In [4]:
def PersonalRank(G, alpha, root):
    '''
    G:是图形结构， 以每个顶点作为键,对应值也是字典，这个嵌套字典的键是链接的节点，值可能是这两个顶点连接的权重值
    '''
    rank = {}
    rank= {x:0 for x in G.keys()}
    rank[root] = 1
    for k in range(20):
        tmp = {x:0 for x in G.keys()}
        for i,ri in G.items():
            for j, wij in ri.items():
                if j not in tmp:
                    tmp[j] = 0
                #tmp[j] += 0.6 * rank[i] /float(len(ri))
                tmp[j] += alpha * rank[i] /float(len(ri))
                if j == root:
                    tmp[j] += 1 - alpha
        rank = tmp
    return rank